In [17]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
import os
import timeit


# Building the Soft-SVM

In [2]:
def gradient_comp(X,y,C,w0):
    summands_of_X = X[X.dot(w0) * y <= 1]
    summands_of_y = y[summands_of_X.index]
#     Y = []
#     for i in range(summands_of_X.shape[1]):
#         Y.append(list(summands_of_y))
#     summands = -1*np.multiply(pd.DataFrame(Y).values.T,summands_of_X)
    summands = -1*summands_of_X.multiply(summands_of_y,axis=0)
    gradient = w0 + C*summands.sum()
    return gradient

In [3]:
def stochastic_gradient_comp(X,y,C,w0,batch):
    Xsamp = X.sample(batch)
    ysamp = y[Xsamp.index]
    summands_of_X = Xsamp[Xsamp.dot(w0) * ysamp <= 1]
    summands_of_y = ysamp[summands_of_X.index]
    #Y = []
    #for i in range(summands_of_X.shape[1]):
        #Y.append(list(summands_of_y))
    summands = -1*summands_of_X.multiply(summands_of_y,axis=0)
    #summands = -1*np.multiply(pd.DataFrame(Y).values.T,summands_of_X)
    gradient = w0 + C*(1/batch)*summands.sum()
    return gradient

In [4]:
def soft_SVM_training(X,y,C,w0,eps,lr,n,batch):
    # X is training data
    # y is labels
    # C is penalty of how hard to be
    # w is initial weights vector (np.array)
    # eps is convergence criterion
    # lr is learning rate
    
    # Computing gradient of L
    i = 0
    gradw = stochastic_gradient_comp(X,y,C,w0,batch)
    w1 = w0 - lr*gradw
    while(np.linalg.norm(w0-w1) > eps):
        w0 = w1
        w1 = w0 - lr*gradient_comp(X,y,C,w0)
        i = i + 1
        if(i%10==0):
            print(i)
        if i == n:
            break
    print(np.linalg.norm(w0-w1))
    return w1

In [5]:
def Testing_soft_SVM(X,y,w):
    # X is the X_test set (in the form of the training set). There should
    # be a column of ones at the same spot as there is in the training.
    # y is the y_test actual values
    # w is the out put weights from the soft_SVM_training
    vals = X.values.dot(w)
    predictions = pd.Series((vals/abs(vals)).astype(int))
    TP = sum(y[(predictions[(predictions > 0)].index)] > 0)
    TN = sum(y[predictions[(predictions < 0)].index] < 0)
    FP = sum(y[predictions[(predictions > 0)].index] < 0)
    FN = sum(y[predictions[(predictions < 0)].index] > 0)
#     print("TP",TP)
#     print("TN",TN)
#     print("FP",FP)
#     print("FN",FN)
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    #precision = TP/(TP+FP)
    #recall = TP/(TP+FN)
    #f1 = 2*(precision*recall)/(precision+recall)
    #return {"accuracy":accuracy,"precision":precision,"recall":recall, "f1":f1}
    #return {"accuracy":accuracy,"recall":recall}
    X.drop("ones",axis=1)
    return accuracy

In [6]:
def produce_confusion_matrix(y_pred, y_test):
    res = pd.DataFrame({"Prediction": y_pred, "Actual": y_test})

    res_positives = res[res["Actual"] == 1]
    res_negatives = res[res["Actual"] == -1]

    positives_dict = dict(res_positives["Prediction"].value_counts())
    TPs = positives_dict[1]
    FNs = positives_dict[-1]

    negatives_dict = dict(res_negatives["Prediction"].value_counts())
    TNs = negatives_dict[-1]
    FPs = negatives_dict[1]

    positives = pd.Series([TPs, FNs])
    negatives = pd.Series([FPs, TNs])

    confusion_matrix = pd.DataFrame({"Actually Positive": positives, "Actually Negative": negatives})
    confusion_matrix.rename({0: "Prediced Positive", 1: "Predicted Negative"}, inplace=True)
    return confusion_matrix

# Train

In [7]:
os.chdir("..")

In [8]:
training = pd.read_csv("tf_idf_train2.csv")

In [9]:
training = training.drop(["Unnamed: 0"],axis=1)
training.head()

,life,comedy,whole,reality,cartoon,good,also,stories,behaved,cross,...,amazons-dr,now-over,creature-basically,mud-wrestling,multi-sexual,barabarian,gamorrean,d-grade,highly-entertaining,Label
0,1.734095,2.38033,2.250752,3.455865,4.257334,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.000000,2.38033,0.000000,0.000000,0.000000,2.900542,2.707422,3.329807,7.082109,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.675593,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.353711,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [10]:
testing = pd.read_csv("tf_idf_test2.csv").drop(["Unnamed: 0"],axis=1)
testing.head()

,good,well,comedy,overcome,coaxed,also,love,something,must,tv,...,aggrandizement,uneventfully,feng,satans,day-for-night,preppie,dines,bleibtreu,terminating,Label
0,0.997067,2.532548,2.375586,5.330841,8.740337,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.997067,1.266274,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.000000,3.798821,0.000000,0.000000,0.000000,1.379756,1.735455,1.821889,2.212013,2.49423,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.997067,3.798821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [11]:
intersection = list(set(training.columns).intersection(set(testing.columns)))

In [12]:
X_train = training[intersection].drop("Label",axis=1)
X_train["ones"] = [1]*len(X_train)
y_train = training["Label"]

In [13]:
X_test = testing[intersection].drop("Label",axis=1)
X_test["ones"] = [1]*len(X_test)
y_test = testing["Label"]

In [14]:
X_train.head()

,sweater,yugoslavia,strengthened,pickins,mocking,ol,mean-spirited,pepper,insoluble,punch-drunk,...,prohibitions,sobbed,incarnate,parasitic,deceitfulness,liked,alvarado,woodchuck,thunderstorm,ones
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [15]:
X_test.head()

,sweater,yugoslavia,strengthened,pickins,mocking,ol,mean-spirited,pepper,insoluble,punch-drunk,...,prohibitions,sobbed,incarnate,parasitic,deceitfulness,liked,alvarado,woodchuck,thunderstorm,ones
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.990944,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1


In [19]:
start_time = timeit.default_timer()

w = soft_SVM_training(X_train,
                  y_train,
                  30,
                  np.array(X_train.mean()),
                  10**-6,
                  0.001,
                  200,
                  128)
acc = Testing_soft_SVM(X_test,y_test,w)
print("Accuracy:",acc)
elapsed = timeit.default_timer() - start_time
print("Time (minutes) elapsed for this cell:", elapsed/60)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
460.59006577053896
Accuracy: 0.67372
Time (minutes) elapsed for this cell: 6.134175799733202


In [20]:
vals = X_test.values.dot(w)
predictions = pd.Series((vals/abs(vals)).astype(int))
confusion_matrix = produce_confusion_matrix(predictions, y_test)
confusion_matrix

,Actually Positive,Actually Negative
Prediced Positive,11123,6780
Predicted Negative,1377,5720


In [21]:
start_time = timeit.default_timer()

w = soft_SVM_training(X_train,
                  y_train,
                  150,
                  np.array(X_train.mean()),
                  10**-6,
                  0.001,
                  750,
                  128)
acc = Testing_soft_SVM(X_test,y_test,w)
print("Accuracy:",acc)
elapsed = timeit.default_timer() - start_time
print("Time (minutes) elapsed for this cell:", elapsed/60)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
1380.4472429473115
Accuracy: 0.67548
Time (minutes) elapsed for this cell: 12.496814970683287


In [22]:
vals = X_test.values.dot(w)
predictions = pd.Series((vals/abs(vals)).astype(int))
confusion_matrix = produce_confusion_matrix(predictions, y_test)
confusion_matrix

,Actually Positive,Actually Negative
Prediced Positive,10564,6177
Predicted Negative,1936,6323
